# 유튜버 크롤링 후 DB 적재

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import numpy as np
import psycopg2
import re

In [2]:
# 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

# 해시태그 함수
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        print(hash_tag)
        hashtag_list.append(hash_tag)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

In [35]:
#업로더 검색
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

#업로더 홈페이지 접속
url = 'https://www.youtube.com/c/%EB%94%B0%EB%9C%BB%ED%95%9C%EC%B1%85%ED%95%9C%EC%9E%94%EC%98%A4%EB%94%94%EC%98%A4%EB%B6%81/videos'
driver.get(url)

driver.maximize_window()

멜로우, 배배자매, 깡스타일리스트, 안녕하새은, 핏더사이즈, 짱구대디, 혜인, 송이송이, 미니월드, samedifference, 오정규

#카테고리 리스트
keyword_list = ['조명', '무드등', '형광등', '방등', '거실등', \
    '침구', '매트리스', '블라인드', '커튼', '러그', '매트', '쿠션', '소파', \
    '테이블', '식탁', '책상', '거실장', '서랍', '진열장', '책장', '선반', '행거', '옷장', '의자', '화장대', '거울']

한주의책이야기, 유튜북진서, 한뼘서재, 따뜻한책한잔오디오북

In [36]:
# 제목 & 링크 크롤링

# 스크롤 다운 20번 실행
body = driver.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)

for i in range(1,10):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = [] #영상번호
name_list = [] #영상제목
url_list = [] #영상주소

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())

for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

print('영상 개수: ', len(name_list))

영상 개수:  159


In [37]:
# 업로드일 & 해시태그 가져오기
day_list = [] #업로드일
hashtag_list = [] #해시태그

for i in range(0,len(name_list)):
    
    if url_list[i].split('/')[3] == 'shorts':  #쇼츠

        print(i+1, ': 쇼츠', end=' ')

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        driver.get(url_list[i])

        time.sleep(2)

        #더보기 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
        time.sleep(2)

        #설명 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
        time.sleep(2)

        #업로드일 정보 가져오기
        day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()

        print(day)
        day_list.append(day)

        time.sleep(2) 

        # 해시태그 가져오기
        outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
        get_hashtag(outline_test)

    else :
        print(i+1, ': 일반', end=' ')

        driver.get(url_list[i])
            
        time.sleep(2)    

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        time.sleep(2)  

        #업로드일 정보 가져오기
        #day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
        day = driver.find_element_by_id("info-strings").text.split('\n')[0]
        
        print(day)
        day_list.append(day)

        time.sleep(2) 

        # 해시태그 가져오기
        outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
        get_hashtag(outline_test)

print('끝')

1 : 일반 2022. 6. 9.
#오십에읽는논어 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #고전 #공자 #인문학
2 : 일반 2022. 6. 7.
#나는남편에게아파트를선물했다 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #재테크
3 : 일반 2022. 6. 5.
#시크릿 #론다번 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #부자마인드 #자기계발
4 : 일반 2022. 6. 2.
#부자가되는 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #부자마인드 #자기계발
5 : 일반 2022. 5. 31.
#세븐 #전인구 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #재테크
6 : 일반 2022. 5. 29.
#매직 #시크릿 #론다번 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #자기계발
7 : 일반 2022. 5. 26.
#트리거 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #자기계발
8 : 일반 2022. 5. 24.
#머니체인저 #재테크 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰
9 : 일반 2022. 5. 22.
#시크릿더파워 #시크릿 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #자기계발 #성공학
10 : 일반 2022. 5. 19.
#부의역발상 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #자기계발 #성공학
11 : 일반 2022. 5. 17.
#보도섀퍼의이기는습관 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #자기계발 #성공학
12 : 일반 2022. 5. 15.
#당신도부자가될수있다 #책읽어주는여자 #오디오북 #따뜻한책한잔 #출근할때듣는 #자기전에듣는 #책리뷰 #재테크 #부자되는법
1

In [38]:
day_list2 = [] #전처리 후 업로드일

#업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

#df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

# 업로드한 지 1년 이상된 데이터는 제거함
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])
youtubeDf = youtubeDf[youtubeDf['업로드일'] > '2021-06-01']

youtubeDf.tail()

,제목,업로드일,주소,해시태그
133,딸에게 보내는 심리학 편지 / 오디오북 / 56분 / 책읽어주는여자 / 따뜻한책한잔,2021-06-23,https://www.youtube.com/watch?v=oBbe1rUiVjw,#따뜻한책한잔 #책읽어주는여자 #출근할때듣는 #자기전에듣는 #오디오북 #딸에게보내는...
134,부의 본능 / 부자가 되려면 이것만은 버려라 / 오디오북 / 50분 / 책읽어주는여...,2021-06-17,https://www.youtube.com/watch?v=Nl4Ff17ctC8,#따뜻한책한잔 #책읽어주는여자 #출근할때듣는 #자기전에듣는 #오디오북 #부의본능 #제테크
135,한계는 내 머릿속에만 있다 / 자기계발의 정석 / 오디오북 / 35분 / 책읽어주는...,2021-06-14,https://www.youtube.com/watch?v=4MEU82iHlNU,#따뜻한책한잔 #책읽어주는여자 #출근할때듣는 #자기전에듣는 #오디오북 #한계는내머릿...
136,내 마음 다치지않게 / 관계에 지친 당신 / 오디오북 / 24분 / 책읽어주는여자 ...,2021-06-08,https://www.youtube.com/watch?v=Qsa0vxyIgPw,#따뜻한책한잔 #책읽어주는여자 #출근할때듣는 #자기전에듣는 #오디오북 #내마음다치지...
137,부의 추월차선 / 책읽어주는여자 / 오디오북 / 56분 / 빠르게부자되기 / 따뜻한책한잔,2021-06-05,https://www.youtube.com/watch?v=VJlKxOXFaY0,#따뜻한책한잔 #책읽어주는여자 #출근할때듣는 #자기전에듣는 #오디오북 #부의추월차선...


In [39]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_books'

#table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

#db 연결 종료
conn.cursor().close()
conn.close()

# #############여기까지만#############

In [16]:
youtubeDf.to_csv('C:\jupiter_workspace\zam_project\outputs/유튜브_'+keyword + '.csv', encoding='utf-8-sig', index=False)

In [12]:
# 댓글 크롤링
video_num = 0
video_idex=[]
video_comm = []


for i in range(len(name_list)):
    driver.get(url_list[i])
    
    time.sleep(3)
    
    try :
        # 동영상 재생 멈추기 
        body = driver.find_element_by_tag_name('body')
        body.send_keys(Keys.SPACE)
    except: # 스페이스 누르기 안되면 직접 일시중지 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[1]/div/div/div/ytd-player/div/div/div[1]/video').click()
        
    # 유튜브 프리미엄 팝업창 닫기
    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass

    # 댓글 보이게 하기(밑으로 내리기)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,800);")
    time.sleep(8)
    
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(5)
    
        
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    comment_list = soup.select("yt-formatted-string#content-text")

    comment_final = []

    for i in range(len(comment_list)):
        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        temp_comment = temp_comment.replace('\xa0@', '')
        temp_comment = temp_comment.replace('\xa0', '')
        temp_comment = temp_comment.replace('\r', '')
        comment_final.append(temp_comment)
    
    video_idex.append(video_num)
    video_comm.append(comment_final)
    video_num += 1


KeyboardInterrupt: 

In [23]:
df = []
for i in range(len(video_idex)):
    if len(video_comm[i]) != 0:
        n = np.full((len(video_comm[i])),i+1)
        temp = pd.DataFrame([ x for x in zip(n,video_comm[i])])
        df.append(temp)

In [24]:
comment_df = df[0]
for i in range(1, len(df)):
    comment_df = pd.concat([comment_df, df[i]])
    
comment_df.columns = ['영상 번호', '댓글']
comment_df.head()

,영상 번호,댓글
0,1,이런 컨텐츠 좋은 거 같아요~몰랐던 브래드도 알고 메인템도 알아서 좋네요~
1,1,각 브랜드에서 매력있는 아이템들이 많은걸 알아서 좋았네요 ㅎㅎ
2,1,형 이런 컨셉 유료광고면 한달에 3번 이상 유료광고 받아도 볼만한 느낌… 짱이다 형️
3,1,마지막 바지 예뻐서 사려고봤더니 품절이네요 ㅠㅠ 재입고되면 바로사야지 !
4,1,와 데님셔츠는 가격대비 미치긴한듯


In [11]:
#CSV형태로 저장하기
youtubeDf.to_csv('./'+'유튜브_핏더사이즈_크롤링.csv', encoding='utf-8-sig', index=False)
comment_df.to_csv('./'+'유튜브_핏더사이즈_댓글.csv', encoding='utf-8-sig', index=False)